In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

%matplotlib inline

In [13]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-563a1c91-40ed-4556-bbe7-0e1328453437',
    'IBM_API_KEY_ID': 'oqglYgpciJdM7HrcoaDfb86whTUtJW38kg6WQuCBI_DW',
    'ENDPOINT': 'https://s3.ap-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'titanicdatarefineryampautoai-donotdelete-pr-xsqfuzgdbrjtti',
    'FILE': 'titanic.csv'
}


In [14]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_855afe22c6b14707b1bf23e1fcd64748 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='oqglYgpciJdM7HrcoaDfb86whTUtJW38kg6WQuCBI_DW',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_855afe22c6b14707b1bf23e1fcd64748.get_object(Bucket='datarefineryex1titanic-donotdelete-pr-5bne67qqudo7xb', Key='titanic2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 
df_titanic = pd.read_csv(streaming_body_1)
df_titanic.head(10)

ClientError: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied

In [10]:
df_titanic.info()
df_titanic.describe()

NameError: name 'df_titanic' is not defined

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,6))

df_titanic['survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.2f%%',ax=ax[0])
ax[0].set_title('Survived')
ax[0].set_ylabel('')

sns.countplot('survived',data=df_titanic,ax=ax[1])
ax[1].set_title('Survived')
plt.show()

In [ ]:
df_titanic['survived'].value_counts()

In [ ]:
df_titanic['age'].hist(bins=20,figsize=(18,8),grid=False);

In [ ]:
df_titanic['age_cat'] = pd.cut(df_titanic['age'], bins=[0, 10, 20, 50, 100], 
                             include_lowest=True, labels=['baby', 'teenage', 'adult', 'old'])
plt.figure(figsize=[12,4])
plt.subplot(131)
sns.barplot('pclass', 'survived', data=df_titanic)
plt.subplot(132)
sns.barplot('age_cat', 'survived', data=df_titanic)
plt.subplot(133)
sns.barplot('sex', 'survived', data=df_titanic)
plt.subplots_adjust(top=1, bottom=0.1, left=0.10, right=1, hspace=0.5, wspace=0.5)
plt.show()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,6))
sns.countplot('sex',data=df_titanic, ax=ax[0])
ax[0].set_title('Count of Passengers by Sex')

sns.countplot('sex',hue='survived',data=df_titanic, ax=ax[1])
ax[1].set_title('Sex:Survived vs Dead')
plt.show()

In [ ]:
df_titanic.isnull().sum()

In [ ]:
# null이 실제로 몇 번 row에 있는지 확인한다
print("Rows where fare value is missing:")
df_titanic[df_titanic['fare'].isnull()]

In [ ]:
print("Rows where age value is missing:")
df_titanic[df_titanic['age'].isnull()]

In [ ]:
# 중요! - 데이터 정제2 - 컬럼값이 NULL이 아닌 것만 필터링
df_titanic = df_titanic[df_titanic['age'].notna()]
df_titanic = df_titanic[df_titanic['fare'].notna()]
df_titanic.isnull().sum()

In [ ]:
print("Rows where fare value is missing:")
df_titanic[df_titanic['fare'].isnull()]

In [ ]:
print("Rows where age value is missing:")
df_titanic[df_titanic['age'].isnull()]

In [ ]:
df_titanic.sex.dtype

In [ ]:
# sex를 문자열에서 정수로 변경하는 함수를 정의. LabelEncoder.fit()은 알파벳의 순서로 sort해서 encoding한다
from sklearn.preprocessing import LabelEncoder
def sex_to_int(data):
    le = LabelEncoder()
    le.fit(["male","female"])
    data["sex"]=le.transform(data["sex"]) 
    return data

In [ ]:
# sex를 문자열에서 정수로 변경하는 함수를 정의
#df_titanic = sex_to_int(df_titanic)
#df_titanic.sex.dtype

In [ ]:
x_data = df_titanic.values[:, [0,3,4,5,6,8]]
y_data = df_titanic.values[:, [1]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=7)

In [ ]:
x_data

In [ ]:
y_data

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.core import Dense
np.random.seed(7)

print('tensorflow version : ', tf.__version__)
print('keras version : ', keras.__version__)

In [ ]:
model = Sequential()
model.add(Dense(255, input_shape=(6,), activation='relu'))
model.add(Dense((1), activation='sigmoid'))
model.compile(loss='mse', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [15]:
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

plt.figure(figsize=(12,8))
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.legend(['loss','val_loss', 'acc','val_acc'])
plt.show()

NameError: name 'model' is not defined

In [ ]:
dicaprio = np.array([3., 1., 19., 0., 0., 5.]).reshape(1,6)
winslet = np.array([1., 0., 17., 1., 2., 100.]).reshape(1,6)

In [ ]:
model.predict(dicaprio)

In [ ]:
model.predict(winslet)